# Testing the Pipeline from Class

In [278]:
import json
import pandas as pd
import matplotlib.pyplot as plt


from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import TreebankWordTokenizer
from nltk.stem import PorterStemmer

from pipeline import NLPPipe

from helper_functions import txt_to_df

import pickle
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [299]:
hey = txt_to_df('twitter_data.txt')

In [302]:
corpus_list = hey['long_text'].tolist()

In [303]:
nlp = NLPPipe(vectorizer=CountVectorizer(), 
              tokenizer=TreebankWordTokenizer().tokenize, 
              stemmer=PorterStemmer())

In [305]:
nlp.fit(corpus_list)

In [310]:
nlp.vectorizer.vocabulary_

{'how': 1443,
 'can': 517,
 'you': 3208,
 'possibl': 2255,
 'organ': 2098,
 'for': 1154,
 'polit': 2242,
 'revolut': 2432,
 'if': 1474,
 'even': 1014,
 'chang': 569,
 'your': 3211,
 'dietari': 849,
 'habit': 1321,
 'sacrific': 2469,
 'social': 2639,
 'tast': 2795,
 'bud': 476,
 'go': 1260,
 'vegan': 2999,
 'is': 1536,
 'first': 1122,
 'step': 2704,
 'mani': 1816,
 'peopl': 2168,
 'to': 2876,
 'realiz': 2370,
 'the': 2827,
 'power': 2262,
 'their': 2832,
 'individu': 1503,
 'action': 131,
 'hold': 1418,
 'agre': 162,
 'thi': 2849,
 'too': 2891,
 'so': 2636,
 'confus': 674,
 'wit': 3138,
 'what': 3111,
 'stand': 2690,
 'should': 2580,
 'take': 2785,
 'though': 2857,
 'question': 2334,
 'animalactivist': 215,
 'amp': 203,
 'say': 2498,
 'trump2020': 2928,
 'ok': 2064,
 'more': 1932,
 'of': 2054,
 'cut': 767,
 'endangeredspeciesact': 977,
 'drill': 912,
 'in': 1489,
 'alaska': 177,
 'wildlif': 3130,
 'refug': 2392,
 'vote': 3065,
 'trump': 2927,
 'against': 159,
 'anim': 212,
 'notmypresid

In [237]:
tweet.iloc[0,:]['retweeted_status']['extended_tweet']['full_text']

'how can you possibly organize for a political revolution if you can’t even change your dietary habits? how can you sacrifice for social change if you can’t sacrifice your taste buds? going vegan is a first step for many people to realize the power their individual actions hold.'

In [239]:
tweet.columns

Index(['created_at', 'id', 'id_str', 'text', 'source', 'truncated',
       'in_reply_to_status_id', 'in_reply_to_status_id_str',
       'in_reply_to_user_id', 'in_reply_to_user_id_str',
       'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place',
       'contributors', 'retweeted_status', 'is_quote_status', 'quote_count',
       'reply_count', 'retweet_count', 'favorite_count', 'entities',
       'favorited', 'retweeted', 'filter_level', 'lang', 'timestamp_ms',
       'quoted_status_id', 'quoted_status_id_str', 'quoted_status',
       'quoted_status_permalink', 'display_text_range', 'extended_tweet',
       'possibly_sensitive', 'extended_entities'],
      dtype='object')

In [261]:
type(tweet.iloc[0,:]['retweeted_status'])

dict

In [260]:
if type(tweet.iloc[0,:]['retweeted_status']) is float:
    print('t')

In [259]:
if type(tweet.iloc[1,:]['retweeted_status']) is float:
    print('f')

f


In [254]:
type(tweet.iloc[1,:]['retweeted_status'])

float

In [ ]:
def ext_rt(row):
    if type(row['retweeted_status']) == dict:
        return = tweet.iloc[0,:]['retweeted_status']['extended_tweet']['full_text']


In [266]:
tweet.iloc[0,:]['retweeted_status']['extended_tweet']['full_text']

'how can you possibly organize for a political revolution if you can’t even change your dietary habits? how can you sacrifice for social change if you can’t sacrifice your taste buds? going vegan is a first step for many people to realize the power their individual actions hold.'

In [296]:
hey = txt_to_df('twitter_data.txt')

In [297]:
hey

,long_text
0,how can you possibly organize for a political ...
1,Agreeing this too 😣I'm so confused wit what st...
2,Question to #AnimalActivists &amp; #Vegan sayi...
4,mainstream veganism is very misconstrued. but ...
6,me:\nmy brain: lets just try n do low fat vega...
...,...
700,"""Vegan Miso Soup"" #food #eat #health #tastyfix..."
701,RT @ParitaKansagra_: Garlic &amp; Tomato Focac...
702,"Linen, Room, Body Mist | Fresh, Calming, Sooth..."
703,Made it to 72 hours in my fast. Longest one I’...


In [298]:
hey.to_pickle("long_tweets.pkl")

In [222]:
hey

,long_text
0,RT @ainsleyikes: how can you possibly organize...
1,Agreeing this too 😣I'm so confused wit what st...
2,RT @MaryJoe38642126: Question to #AnimalActivi...
4,RT @ainsleyikes: mainstream veganism is very m...
6,me:\nmy brain: lets just try n do low fat vega...
...,...
700,"""Vegan Miso Soup"" #food #eat #health #tastyfix..."
701,RT @ParitaKansagra_: Garlic &amp; Tomato Focac...
702,"Linen, Room, Body Mist | Fresh, Calming, Sooth..."
703,Made it to 72 hours in my fast. Longest one I’...


In [122]:
tweet[tweet['lang'] =='en']

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,lang,timestamp_ms,quoted_status_id,quoted_status_id_str,quoted_status,quoted_status_permalink,display_text_range,extended_tweet,possibly_sensitive,extended_entities
0,Fri Oct 30 14:57:54 +0000 2020,1322191002405347328,1322191002405347328,RT @ainsleyikes: how can you possibly organize...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,None,NaN,None,...,en,1604069874751,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Fri Oct 30 14:57:56 +0000 2020,1322191008126234625,1322191008126234625,Agreeing this too 😣I'm so confused wit what st...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,None,NaN,None,...,en,1604069876115,1.322187e+18,1322187412039114754,{'created_at': 'Fri Oct 30 14:43:38 +0000 2020...,"{'url': 'https://t.co/wVW6Shesl4', 'expanded':...",NaN,NaN,NaN,NaN
2,Fri Oct 30 14:57:56 +0000 2020,1322191010781384707,1322191010781384707,RT @MaryJoe38642126: Question to #AnimalActivi...,"<a href=""https://script.google.com/macros/s/AK...",False,NaN,None,NaN,None,...,en,1604069876748,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Fri Oct 30 14:57:59 +0000 2020,1322191022328262656,1322191022328262656,RT @ainsleyikes: mainstream veganism is very m...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,None,NaN,None,...,en,1604069879501,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Fri Oct 30 14:58:14 +0000 2020,1322191083783311360,1322191083783311360,me:\nmy brain: lets just try n do low fat vega...,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,None,NaN,None,...,en,1604069894153,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
700,Fri Oct 30 15:29:32 +0000 2020,1322198963827728386,1322198963827728386,"""Vegan Miso Soup"" #food #eat #health #tastyfix...","<a href=""http://engage.social"" rel=""nofollow"">...",False,NaN,None,NaN,None,...,en,1604071772902,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
701,Fri Oct 30 15:29:33 +0000 2020,1322198966742720515,1322198966742720515,RT @ParitaKansagra_: Garlic &amp; Tomato Focac...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,None,NaN,None,...,en,1604071773597,NaN,NaN,NaN,NaN,NaN,NaN,False,"{'media': [{'id': 1321836360286801920, 'id_str..."
702,Fri Oct 30 15:29:34 +0000 2020,1322198970270175235,1322198970270175235,"Linen, Room, Body Mist | Fresh, Calming, Sooth...","<a href=""https://www.twitter-fu.com"" rel=""nofo...",True,NaN,None,NaN,None,...,en,1604071774438,NaN,NaN,NaN,NaN,NaN,"{'full_text': 'Linen, Room, Body Mist | Fresh,...",False,NaN
703,Fri Oct 30 15:29:45 +0000 2020,1322199016118095875,1322199016118095875,Made it to 72 hours in my fast. Longest one I’...,"<a href=""http://twitter.com/download/iphone"" r...",True,NaN,None,NaN,None,...,en,1604071785369,NaN,NaN,NaN,NaN,NaN,{'full_text': 'Made it to 72 hours in my fast....,NaN,NaN


In [116]:
tweets_data_path = 'twitter_data.txt'
tweets_file = open(tweets_data_path, "r")

In [117]:
tweets_data = []
tweets_file = open(tweets_data_path, "r")
for line in tweets_file:
    try:
        tweet = json.loads(line)
        tweets_data.append(tweet)
    except:
        continue

In [118]:
tweet = pd.DataFrame(tweets_data)

In [110]:
en_tweets = tweet[tweet['lang']=='en']

In [112]:
text_df = en_tweets[['text','truncated','extended_tweet']]

In [223]:
text_df.iloc[0,:]['long_text']

'RT @ainsleyikes: how can you possibly organize for a political revolution if you can’t even change your dietary habits? how can you sacrifi…'

In [219]:
text_df.iloc[597,:]['text']

'Made it to 72 hours in my fast. Longest one I’ve ever done. I’m going to have a microwave vegan lasagna for 360 cal… https://t.co/JdGU1LYyCe'

In [75]:
text_df = tweet_df[['text','truncated','extended_tweet']]

In [199]:
text_df['long_text']= text_df.apply(fix_tweets, axis=1)

In [214]:
text_df.iloc[597,:]['extended_tweet']['full_text']

'Made it to 72 hours in my fast. Longest one I’ve ever done. I’m going to have a microwave vegan lasagna for 360 calories..yeah depression meal.'

In [198]:
def fix_tweets(row):
    if row['truncated'] == True:
        return row['extended_tweet']['full_text']
    else:
        return row['text']

In [58]:
tweet_df['retweeted_status'][0]['extended_tweet']['full_text']

'how can you possibly organize for a political revolution if you can’t even change your dietary habits? how can you sacrifice for social change if you can’t sacrifice your taste buds? going vegan is a first step for many people to realize the power their individual actions hold.'

In [64]:
if tweet_df['truncated'] == True:
    tweet = tweet_df['extended_tweet'][x]['full_text']
else:
    tweet = tweet_df['text']

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,lang,timestamp_ms,quoted_status_id,quoted_status_id_str,quoted_status,quoted_status_permalink,display_text_range,extended_tweet,possibly_sensitive,extended_entities
0,Fri Oct 30 14:57:54 +0000 2020,1322191002405347328,1322191002405347328,RT @ainsleyikes: how can you possibly organize...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,None,NaN,None,...,en,1604069874751,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Fri Oct 30 14:57:56 +0000 2020,1322191008126234625,1322191008126234625,Agreeing this too 😣I'm so confused wit what st...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,None,NaN,None,...,en,1604069876115,1.322187e+18,1322187412039114754,{'created_at': 'Fri Oct 30 14:43:38 +0000 2020...,"{'url': 'https://t.co/wVW6Shesl4', 'expanded':...",NaN,NaN,NaN,NaN
2,Fri Oct 30 14:57:56 +0000 2020,1322191010781384707,1322191010781384707,RT @MaryJoe38642126: Question to #AnimalActivi...,"<a href=""https://script.google.com/macros/s/AK...",False,NaN,None,NaN,None,...,en,1604069876748,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fri Oct 30 14:57:57 +0000 2020,1322191013285404672,1322191013285404672,Go Vegan ❤️,"<a href=""http://twitter.com/download/android"" ...",False,NaN,None,NaN,None,...,es,1604069877345,1.322184e+18,1322183981333295104,{'created_at': 'Fri Oct 30 14:30:00 +0000 2020...,"{'url': 'https://t.co/grRvES0hu5', 'expanded':...",NaN,NaN,NaN,NaN
4,Fri Oct 30 14:57:59 +0000 2020,1322191022328262656,1322191022328262656,RT @ainsleyikes: mainstream veganism is very m...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,None,NaN,None,...,en,1604069879501,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
700,Fri Oct 30 15:29:32 +0000 2020,1322198963827728386,1322198963827728386,"""Vegan Miso Soup"" #food #eat #health #tastyfix...","<a href=""http://engage.social"" rel=""nofollow"">...",False,NaN,None,NaN,None,...,en,1604071772902,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
701,Fri Oct 30 15:29:33 +0000 2020,1322198966742720515,1322198966742720515,RT @ParitaKansagra_: Garlic &amp; Tomato Focac...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,None,NaN,None,...,en,1604071773597,NaN,NaN,NaN,NaN,NaN,NaN,False,"{'media': [{'id': 1321836360286801920, 'id_str..."
702,Fri Oct 30 15:29:34 +0000 2020,1322198970270175235,1322198970270175235,"Linen, Room, Body Mist | Fresh, Calming, Sooth...","<a href=""https://www.twitter-fu.com"" rel=""nofo...",True,NaN,None,NaN,None,...,en,1604071774438,NaN,NaN,NaN,NaN,NaN,"{'full_text': 'Linen, Room, Body Mist | Fresh,...",False,NaN
703,Fri Oct 30 15:29:45 +0000 2020,1322199016118095875,1322199016118095875,Made it to 72 hours in my fast. Longest one I’...,"<a href=""http://twitter.com/download/iphone"" r...",True,NaN,None,NaN,None,...,en,1604071785369,NaN,NaN,NaN,NaN,NaN,{'full_text': 'Made it to 72 hours in my fast....,NaN,NaN


In [65]:
tweet_df.columns

Index(['created_at', 'id', 'id_str', 'text', 'source', 'truncated',
       'in_reply_to_status_id', 'in_reply_to_status_id_str',
       'in_reply_to_user_id', 'in_reply_to_user_id_str',
       'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place',
       'contributors', 'retweeted_status', 'is_quote_status', 'quote_count',
       'reply_count', 'retweet_count', 'favorite_count', 'entities',
       'favorited', 'retweeted', 'filter_level', 'lang', 'timestamp_ms',
       'quoted_status_id', 'quoted_status_id_str', 'quoted_status',
       'quoted_status_permalink', 'display_text_range', 'extended_tweet',
       'possibly_sensitive', 'extended_entities'],
      dtype='object')

In [54]:
tweet_df[['retweeted_status']].apply(lambda x: x['extended_tweet']['full_text']
#['retweeted_status'][0]['extended_tweet']['full_text']

'how can you possibly organize for a political revolution if you can’t even change your dietary habits? how can you sacrifice for social change if you can’t sacrifice your taste buds? going vegan is a first step for many people to realize the power their individual actions hold.'

In [29]:
en_tweets = tweet_df[tweet_df['lang']=='en']['text']

In [18]:
nlp = NLPPipe(vectorizer=CountVectorizer(), 
              tokenizer=TreebankWordTokenizer().tokenize, 
              stemmer=PorterStemmer())

In [30]:
nlp.fit(en_tweets)

In [34]:
nlp.transform(en_tweets).toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [32]:
en_tweets

0      RT @ainsleyikes: how can you possibly organize...
1      Agreeing this too 😣I'm so confused wit what st...
2      RT @MaryJoe38642126: Question to #AnimalActivi...
4      RT @ainsleyikes: mainstream veganism is very m...
6      me:\nmy brain: lets just try n do low fat vega...
                             ...                        
700    "Vegan Miso Soup" #food #eat #health #tastyfix...
701    RT @ParitaKansagra_: Garlic &amp; Tomato Focac...
702    Linen, Room, Body Mist | Fresh, Calming, Sooth...
703    Made it to 72 hours in my fast. Longest one I’...
704    RT @FFOODFESS: fess vegan raw chocolate cake w...
Name: text, Length: 599, dtype: object

In [38]:
en_tw_df = pd.DataFrame(en_tweets)

In [39]:
en_tw_df.to_pickle("en_tweets.pkl")